<a href="https://colab.research.google.com/github/ahmedsaadalvi/AI-Trader/blob/main/AI_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <-- Step 1: Download required dependencies
!pip install tensorflow-gpu

!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

     |████████████████████████████████| 497.5 MB 23 kB/s 
     |████████████████████████████████| 462 kB 62.1 MB/s 
     |████████████████████████████████| 109 kB 5.2 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
#@title <-- Step 2: Choose Stock Name and Execute { run: "auto", vertical-output: true }
class AI_Trader():
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.inventory = []
    self.model_name = model_name
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.1
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      target = self.model.predict(state)
      target[0][action] = reward
      self.model.fit(state, target, epochs = 1, verbose = 0)
    if self.epsilon > self.epsilon_final:
      self.epsilon = self.epsilon * self.epsilon_decay

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "-$ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset["Close"]
  return close

def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  if starting_id >= 0:
    windowed_data = data[starting_id: timestep + 1]
  else:
    windowed_data = -starting_id * [data[0]] + list(data[0: timestep + 1])
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
  return np.array([state])

Stock_Name = "AMZN" #@param["AAPL", "AMZN", "INTC", "NVDA", "BAC", "AAL", "T"]
stock_name = Stock_Name
data = dataset_loader(stock_name)
data

Date
2017-03-27     846.820007
2017-03-28     856.000000
2017-03-29     874.320007
2017-03-30     876.340027
2017-03-31     886.539978
                 ...     
2022-03-18    3225.010010
2022-03-21    3229.830078
2022-03-22    3297.780029
2022-03-23    3268.159912
2022-03-23    3268.159912
Name: Close, Length: 1259, dtype: float64

In [ ]:
#@title <-- Step 3: Execute to start analyzation
Analyzation_Counts = 1 #@param{type:"slider", min: 1, max: 10, step: 1}
window_size = 10
episodes = Analyzation_Counts
batch_size = 32
data_samples = len(data) - 1

trader = AI_Trader(window_size)

print("\n\n\n\n\nStock Name: {}".format(stock_name))
for episode in range(1, episodes + 1):
  print("\nAnalyzation Count: {}/{}\n".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    if action == 1:
      trader.inventory.append(data[t])
      print("Bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0:
      buy_price = trader.inventory.pop(0)
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("Sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
    if t == data_samples - 1:
      done = True
    else:
      done = False
    trader.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
      print("\n\n########################\n")
      print("TOTAL PROFIT ESTIMATED: {}".format(total_profit))
      print("\n########################\n\n")
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)







Stock Name: AMZN

Analyzation Count: 1/1



  0%|          | 0/1258 [00:00<?, ?it/s]

Bought:  $ 846.820007
Bought:  $ 856.000000
Sold:  $ 874.320007  Profit: $ 27.500000
Sold:  $ 876.340027  Profit: $ 20.340027
Bought:  $ 891.510010
Sold:  $ 906.830017  Profit: $ 15.320007
Bought:  $ 894.880005
Bought:  $ 907.039978
Sold:  $ 902.359985  Profit: $ 7.479980
Bought:  $ 896.229980
Sold:  $ 903.780029  Profit: -$ 3.259949
Sold:  $ 902.059998  Profit: $ 5.830017
Bought:  $ 907.409973
Bought:  $ 907.619995
Bought:  $ 909.289978
Bought:  $ 918.380005
Bought:  $ 946.940002
Bought:  $ 941.030029
Sold:  $ 937.530029  Profit: $ 30.120056
Bought:  $ 934.150024
Sold:  $ 952.820007  Profit: $ 45.200012


  3%|▎         | 33/1258 [00:10<06:44,  3.03it/s]

...

 33%|███▎      | 419/1258 [28:21<1:00:33,  4.33s/it]

Sold:  $ 1516.729980  Profit: -$ 114.440063


 33%|███▎      | 420/1258 [28:25<1:00:20,  4.32s/it]

Sold:  $ 1502.060059  Profit: -$ 96.949951


 33%|███▎      | 421/1258 [28:30<1:01:23,  4.40s/it]